# #0

In [164]:
# ---- ¿Que año, mes y # de iteraciones quieres correr? ----
anyo = '2020'
mes = 'Abril'
iters = 100000

In [165]:
# ---- Librerías ---- 
import pandas as pd 
import numpy as np
import csv 
import warnings
warnings.filterwarnings('ignore')

In [166]:
# ---- Método MonteCarlo ---- 
def MonteCarlo(population, sample_number, weights_column, 
                        current_distint_names, iterations):
    
    sum_distint_names = 0 #numero de apellidos distintos
    higher = 0       #veces que el # de apellidos simulado fue mayor al actual
    lower = 0        #veces que el # de apellidos simulado fue menor al actual
    equal = 0        #veces que el # de apellidos simulado fue igual al actual
    
    
    for i in range(0, iterations): #numero de iteraciones
        
        #esto saca un random example de la poblacion (general), le damos un "N" y un peso específico a cada apellido
        random_weighted_sample = population.sample(n = sample_number, replace = True, 
                                                   weights= weights_column, axis = 0)
        
        #sacamos el numero de apellidos distintos simulados
        distint_names = len(set(random_weighted_sample.index)) #Assuming index contains surnames
        
        
        sum_distint_names += distint_names
        
        
        if distint_names > current_distint_names:
            
            higher += 1

        elif distint_names < current_distint_names:
            
            lower += 1
        
        else:
            
            equal += 1
         
    #guardar output
    output = {'Employees': sample_number,
              'AVG Surnames': sum_distint_names/iterations, 
              'Current Surnames': current_distint_names, 
              '# Simulated > Current': higher, 
              '# Simulated < Current': lower, 
              '# Simulated = Current': equal, 
              'Iterations': iterations
             }
    
    return output

In [167]:
# ---- DB Apellidos Paternos ---- 
df_paternos = pd.read_excel('data publica/apellidos_paternos.xlsx', index_col=0, usecols = 'A:D',engine="openpyxl")

In [168]:
# ---- DB Instituciones ---- 
df_instituciones = pd.read_csv('data publica/Acronimos Organismos.csv', index_col=0, encoding="ISO-8859-1", delimiter =';')

In [169]:
# ---- DB Employees ---- 
df_whole = pd.read_csv('data publica/unidos/'+ mes + anyo + '.csv', index_col=0)

In [170]:
## ---------- Remover tildes de los apellidos y pasar a mayuscula ---------- ##
cols = df_paternos.select_dtypes(include=[np.object]).columns
df_paternos[cols] =df_paternos[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
cols = df.select_dtypes(include=[np.object]).columns
df[cols] =df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df['Paterno'] = df['Paterno'].str.upper()

In [171]:
## ---------- Apellidos Paternos x Región ---------- ##
pd.options.display.float_format = '{:,.2f}'.format 
df_paternos['AP PATERNO'] = df_paternos['AP PATERNO'].str.upper()
df_paternos_region = df_paternos[df_paternos['AP PATERNO'].notna()]
df_paternos_region = df_paternos_region.groupby(['REG','AP PATERNO']).agg({'CANTIDAD': "sum"}).sort_values(by=['CANTIDAD'], ascending=False)
df_paternos_region = df_paternos_region[df_paternos_region['CANTIDAD'] > 1]
df_paternos_region['CANTIDAD'] = df_paternos_region['CANTIDAD'].astype(int)
## ---------- Apellidos Paternos en Chile ---------- ##
df_paternos_region = df_paternos_region.reset_index(level=['REG'])
df_paternos_total = df_paternos_region.groupby(['AP PATERNO']).agg({'CANTIDAD': "sum"}).sort_values(by=['CANTIDAD'], ascending=False)
df_paternos_total['PORCENTAJE'] = (df_paternos_total['CANTIDAD']/df_paternos_total['CANTIDAD'].sum())
df_paternos_total['CANTIDAD'] = df_paternos_total['CANTIDAD'].astype(int)
df_paternos_total = df_paternos_total.dropna()
df_paternos_total = df_paternos_total[df_paternos_total['CANTIDAD'] > 1]

In [172]:
df_paternos_total

,CANTIDAD,PORCENTAJE
AP PATERNO,,
GONZALEZ,444542,0.02
MUNOZ,349675,0.02
ROJAS,249660,0.01
DIAZ,246901,0.01
PEREZ,196733,0.01
...,...,...
CANOBA,2,0.00
CANNY,2,0.00
CANNIZZO,2,0.00


In [173]:
region_codes = {'Region Metropolitana de Santiago':13,
                'Region de Antofagasta':2,
                'Region de Arica y Parinacota':15,
                'Region de Atacama':3,
                'Region de Aysen':11,
                'Region de Coquimbo':4,
                'Region de La Araucania':9,
                'Region de Los Lagos':10,
                'Region de Los Rios':14,
                'Region de Magallanes':12,
                'Region de Nuble':16,
                'Region de OHiggins':6,
                'Region de Tarapaca':1,
                'Region de Valparaiso':5,
                'Region del Biobio':8,
                'Region del Maule':7}

# #1

In [174]:
#numero de apellidos unicos en el organismo
distint_names_organismo = len(set(df_whole['Paterno'].value_counts().index)) 

#numero de personas en el organismo
total = len(df_whole['Paterno'])

#saca todos los apellidos de la región
population = df_paternos_total

#El peso de los apellidos de la región
population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()

#Ejecutar MonteCarlo
output = MonteCarlo(population, total, 'PORCENTAJE', distint_names_organismo, iters)

In [175]:
output

{'Employees': 681204,
 'AVG Surnames': 23305.76118,
 'Current Surnames': 30317,
 '# Simulated > Current': 0,
 '# Simulated < Current': 100000,
 '# Simulated = Current': 0,
 'Iterations': 100000}

# #2

In [176]:
df_merged = pd.merge(df, df_instituciones, left_on='organismo_codigo', right_on='Codigo_org', how='inner')
areas = set(df_merged['Padre_org'].value_counts().index)

In [177]:
dict_regiones = { 'Defensa Nacional' : 'All' ,
'Vivienda y Urbanismo' : 'All' ,
'Municipios de Tarapacá' : 'Region de Tarapaca' ,
'Municipios de Atacama' : 'Region de Atacama' ,
'Minería' : 'All' ,
'Medio Ambiente' : 'All' ,
'Educación' : 'All' ,
'Transportes y Telecomunicaciones' : 'All' ,
'Asociación Municipal' : 'All' ,
'Salud' : 'All' ,
'Agricultura' : 'All' ,
'Municipios de Los Lagos' : 'Region de Los Lagos' ,
'La Mujer y Equidad de Género' : 'All' ,
'Municipios de R. Metropolitana de Santiago' : 'Region Metropolitana de Santiago' ,
'Deporte' : 'All' ,
'Secretaría General de Gobierno' : 'All' ,
'Centro de Formación Técnica' : 'All' ,
'Bienes Nacionales' : 'All' ,
'Secretaría General de la Presidencia' : 'All' ,
'Universidades' : 'All' ,
'Municipios del Libertador General Bernardo OHiggins' : 'Region de OHiggins' ,
'Interior y Seguridad Pública' : 'All' ,
'Municipios de Aysen del General Carlos Ibáñez del Campo' : 'Region de Aysen' ,
'Defensoría de los Derechos de la Niñez' : 'All' ,
'Energía' : 'All' ,
'Municipios de Antofagasta' : 'Region de Antofagasta' ,
'Justicia' : 'All' ,
'Municipios del Bíobio' : 'Region del Biobio' ,
'Corporaciones Municipales' : 'All' ,
'Municipios de La Araucanía' : 'Region de La Araucania' ,
'Municipios de Valparaíso' : 'All' ,
'Consejo Defensa del Estado' : 'All' ,
'Municipios de Ñuble' : 'All' ,
'Municipios de Arica y  Parinacota' : 'Region de Arica y Parinacota' ,
'Superintendencia de Servicios Sanitarios' : 'All' ,
'Municipios de Magallanes y de la Antártica Chilena' : 'Region de Magallanes' ,
'Economía Fomento y Turismo' : 'All' ,
'Municipios del Maule' : 'Region del Maule' ,
'Desarrollo Social' : 'All' ,
'Presidencia de la República' : 'All' ,
'Municipios de Los Ríos' : 'Region de Los Rios' ,
'Trabajo y Previsión Social' : 'All' ,
'Fundaciones y corporaciones' : 'All' ,
'Culturas y las Artes' : 'All' ,
'Hacienda' : 'All' ,
'Municipios de Coquimbo' : 'Region de Coquimbo' ,
'Instituto Derechos Humanos' : 'All' ,
'Obras Públicas' : 'All' ,
'Ciencia Tecnología e Innovación' : 'All' ,
'Consejo para la Transparencia' : 'All' ,
'Relaciones Exteriores' : 'All'
}

In [178]:
dict_areas = {}

for area in areas:
    
    #Se filtra la planta por area
    df_area = df_merged[df_merged['Padre_org'] == area]    

    #Número de personas en el área
    totalOrg = len(df_area['Paterno'])
    
    #Si el área tiene más de cien personas
    if totalOrg > 30 :
        
        #Top 10%
        distint_names_area = len(set(df_area['Paterno'].value_counts().index))
        total = len(df_area['Paterno'])    
        region = df_area['region']
        regiones = set(region)
        lenght = len(regiones)
        regiones = list(val for val in regiones)

        if lenght == 1:
            region = regiones[0]
            region = unicodedata.normalize("NFKD",region).encode("ascii","ignore").decode("ascii")
            code_region = region_codes[region]  
            population = df_paternos_region[df_paternos_region['REG'] == code_region] 
        
        #Composición toda la poblacion
        else:
            import unicodedata
            population = df_paternos_total

        population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()
        output = MonteCarlo(population, total, 'PORCENTAJE', distint_names_area, iters)
        
        dict_areas[area] = output

In [179]:
dict_areas

{'Defensa Nacional': {'Employees': 15161,
  'AVG Surnames': 2686.98705,
  'Current Surnames': 2821,
  '# Simulated > Current': 8,
  '# Simulated < Current': 99991,
  '# Simulated = Current': 1,
  'Iterations': 100000},
 'Vivienda y Urbanismo': {'Employees': 6298,
  'AVG Surnames': 1577.82768,
  'Current Surnames': 1663,
  '# Simulated > Current': 61,
  '# Simulated < Current': 99933,
  '# Simulated = Current': 6,
  'Iterations': 100000},
 'Municipios de Tarapacá': {'Employees': 1870,
  'AVG Surnames': 789.48575,
  'Current Surnames': 674,
  '# Simulated > Current': 100000,
  '# Simulated < Current': 0,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 'Municipios de Atacama': {'Employees': 4033,
  'AVG Surnames': 892.51955,
  'Current Surnames': 884,
  '# Simulated > Current': 67923,
  '# Simulated < Current': 30073,
  '# Simulated = Current': 2004,
  'Iterations': 100000},
 'Minería': {'Employees': 914,
  'AVG Surnames': 481.85162,
  'Current Surnames': 522,
  '# Simulated > Cur

In [180]:
nombre_archivo = 'resultados/3-{0}.csv'.format(iters)
data = [] 
columns = []

for key, value in dict_areas.items():
    data.append(value)
    columns.append(key)
    
total = pd.DataFrame(data, index = columns) 
total.to_csv(nombre_archivo)

# #3

In [181]:
df_planta = pd.read_csv('data publica/por contrato/'+ mes + anyo + 'Planta.csv', index_col=0)
df_honorarios = pd.read_csv('data publica/por contrato/'+ mes + anyo + 'Honorarios.csv', index_col=0)
df_contrata = pd.read_csv('data publica/por contrato/'+ mes + anyo + 'Contrata.csv', index_col=0)
df_codigotrabajo = pd.read_csv('data publica/por contrato/'+ mes + anyo + 'Codigotrabajo.csv', index_col=0)

In [182]:
surnames_planta = len(set(df_planta['Paterno'].value_counts().index))
total_planta = len(df_planta['Paterno'])
surnames_contrata = len(set(df_contrata['Paterno'].value_counts().index))
total_contrata = len(df_contrata['Paterno'])
surnames_honorarios = len(set(df_honorarios['Paterno'].value_counts().index))
total_honorarios = len(df_honorarios['Paterno'])
surnames_codigotrabajo = len(set(df_codigotrabajo['Paterno'].value_counts().index))
total_codigotrabajo = len(df_codigotrabajo['Paterno'])
population = df_paternos_total
population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()

output_planta = MonteCarlo(population, total_planta, 'PORCENTAJE', surnames_planta, iters)
output_contrata = MonteCarlo(population, total_contrata, 'PORCENTAJE', surnames_contrata, iters)
output_honorarios = MonteCarlo(population, total_honorarios, 'PORCENTAJE', surnames_honorarios, iters)
output_codigotrabajo = MonteCarlo(population, total_codigotrabajo, 'PORCENTAJE', surnames_codigotrabajo, iters)

In [183]:
output_planta, output_contrata, output_honorarios, output_codigotrabajo

({'Employees': 162505,
  'AVG Surnames': 11269.26061,
  'Current Surnames': 12326,
  '# Simulated > Current': 0,
  '# Simulated < Current': 100000,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 {'Employees': 298139,
  'AVG Surnames': 15657.99061,
  'Current Surnames': 19512,
  '# Simulated > Current': 0,
  '# Simulated < Current': 100000,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 {'Employees': 109785,
  'AVG Surnames': 8993.98358,
  'Current Surnames': 12302,
  '# Simulated > Current': 0,
  '# Simulated < Current': 100000,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 {'Employees': 110775,
  'AVG Surnames': 9041.67073,
  'Current Surnames': 10136,
  '# Simulated > Current': 0,
  '# Simulated < Current': 100000,
  '# Simulated = Current': 0,
  'Iterations': 100000})

# #4

In [184]:
#Dictionario con los resultados
dict_regiones = {}

#Para cada organismo
for region in region_codes:
    
    df_region = df[df['region'] == region]

    #numero de apellidos unicos en el organismo
    distint_names_region = len(set(df_region['Paterno'].value_counts().index))

    #numero de personas en el organismo
    total = len(df_region['Paterno'])
        
    import unicodedata
    region = unicodedata.normalize("NFKD",region).encode("ascii","ignore").decode("ascii")

    #codigo de la región
    code_region = region_codes[region]  
    
    #saca todos los apellidos de la región
    population = df_paternos_region[df_paternos_region['REG'] == code_region]
    
    #El peso de los apellidos de la región
    population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()
    
    #Ejecutar MonteCarlo
    output = MonteCarlo(population, total, 'PORCENTAJE', distint_names_region, iters)
    
    #Guardar resultado
    dict_regiones[region] = output


In [185]:
dict_regiones

{'Region Metropolitana de Santiago': {'Employees': 220135,
  'AVG Surnames': 15123.49602,
  'Current Surnames': 13537,
  '# Simulated > Current': 100000,
  '# Simulated < Current': 0,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 'Region de Antofagasta': {'Employees': 22753,
  'AVG Surnames': 2691.58451,
  'Current Surnames': 2841,
  '# Simulated > Current': 0,
  '# Simulated < Current': 100000,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 'Region de Arica y Parinacota': {'Employees': 9885,
  'AVG Surnames': 1571.34047,
  'Current Surnames': 1875,
  '# Simulated > Current': 0,
  '# Simulated < Current': 100000,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 'Region de Atacama': {'Employees': 13361,
  'AVG Surnames': 1529.29652,
  'Current Surnames': 1711,
  '# Simulated > Current': 0,
  '# Simulated < Current': 100000,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 'Region de Aysen': {'Employees': 11405,
  'AVG Surnames': 1413.79023,
  'Current Su

In [186]:
nombre_archivo = 'resultados/4-{0}.csv'.format(numero_iteraciones)
data = [] 
columns = []

for key, value in dict_regiones.items():
    data.append(value)
    columns.append(key)
    
total = pd.DataFrame(data, index = columns) 
total.to_csv(nombre_archivo)

# #5

In [187]:
#Dictionario con los resultados
dict_organismos = {}

#Generar deciles en base a la remuneración bruta
df['decilePlanta'] = pd.qcut(df['remuneracionbruta_mensual'], 10, labels=False, duplicates='drop')
    
#Total de empleados en ese organismo
totalAll = len(df['Paterno'])
    
#Decil Top
topDecil = df[df['decilePlanta'] == 9]
    
#Decil bottom
bottomDecil = df[df['decilePlanta'] == 0]
    
#---------------------------Para el top 10% --------------------------#
        
#Cantidad única de apellidos
distint_names_organismo = len(set(topDecil['Paterno'].value_counts().index))
        
#Cantidad de personas en el decil top
totalTop = len(topDecil['Paterno'])
        
#Población de esa región
population = df_paternos_total 
        
#Sacamos los pesos de cada apellido i.e, el porcentaje en esa región
population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()
        
#Corremos la Monte Carlo
outputTop = MonteCarlo(population, totalTop, 'PORCENTAJE', distint_names_organismo, iters)
        
#---------------------------Para el bottom 10% --------------------------#

#Para el bottom 90%
distint_names_organismo = len(set(bottomDecil['Paterno'].value_counts().index))
        
totalBottom = len(bottomDecil['Paterno'])
                
outputBottom = MonteCarlo(population, totalBottom, 'PORCENTAJE', distint_names_organismo, iters)
        

In [188]:
outputTop, outputBottom

({'Employees': 57141,
  'AVG Surnames': 6082.49902,
  'Current Surnames': 8006,
  '# Simulated > Current': 0,
  '# Simulated < Current': 100000,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 {'Employees': 57142,
  'AVG Surnames': 6082.61031,
  'Current Surnames': 4946,
  '# Simulated > Current': 100000,
  '# Simulated < Current': 0,
  '# Simulated = Current': 0,
  'Iterations': 100000})

In [189]:
#Generar deciles en base a la remuneración bruta
dict_deciles = {}
df['decilePlanta'] = pd.qcut(df['remuneracionbruta_mensual'], 10, labels=False, duplicates='drop')
    
#Población
population = df_paternos_total 
        
#Sacamos los pesos de cada apellido i.e, el porcentaje en esa región
population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()

for i in range (0, 10):
    
    Decil = df[df['decilePlanta'] == i]
    
    #Apellidos unicos en el decil
    distint_names_decil = len(set(Decil['Paterno'].value_counts().index))
        
    #Cantidad de personas en el decil top
    totalDecil = len(Decil['Paterno'])
        
    #Corremos la Monte Carlo
    output = MonteCarlo(population, totalDecil, 'PORCENTAJE', distint_names_decil, iters)
        
    dict_deciles[i] = output

In [190]:
dict_deciles

{0: {'Employees': 57142,
  'AVG Surnames': 6082.34517,
  'Current Surnames': 4946,
  '# Simulated > Current': 100000,
  '# Simulated < Current': 0,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 1: {'Employees': 57260,
  'AVG Surnames': 6089.96622,
  'Current Surnames': 4833,
  '# Simulated > Current': 100000,
  '# Simulated < Current': 0,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 2: {'Employees': 57026,
  'AVG Surnames': 6074.60493,
  'Current Surnames': 4934,
  '# Simulated > Current': 100000,
  '# Simulated < Current': 0,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 3: {'Employees': 57140,
  'AVG Surnames': 6082.21288,
  'Current Surnames': 5158,
  '# Simulated > Current': 100000,
  '# Simulated < Current': 0,
  '# Simulated = Current': 0,
  'Iterations': 100000},
 4: {'Employees': 57142,
  'AVG Surnames': 6082.56726,
  'Current Surnames': 5392,
  '# Simulated > Current': 100000,
  '# Simulated < Current': 0,
  '# Simulated = Current': 0,
  'Iterati